In [1]:
import matplotlib.pyplot as plt
from PIL import Image, ImageSequence
import numpy as np

In [12]:

def separate_channels(gif_path):
    # Load GIF
    with Image.open(gif_path) as img:
        frames = [frame.copy() for frame in ImageSequence.Iterator(img)]
    
    # Separate blue and yellow channels
    blue_frames = [frame.convert('RGB').split()[2] for frame in frames]
    yellow_frames = [(np.array(frame.convert('RGB').split()[1]) 
                      + np.array(frame.convert('RGB').split()[0])) //2 for frame in frames]
    yellow_frames = [Image.fromarray(yellow_frame) for yellow_frame in yellow_frames]
    
    return blue_frames, yellow_frames

def apply_inferno_colormap(frames):
    inferno = plt.cm.get_cmap("inferno")
    np_color_frames = []
    for frame in frames:
        np_frame = np.array(frame)
        np_color_frames.append(inferno(np_frame)[:, :, :3])
    colored_frames = [Image.fromarray(np.uint8(np_color_frame * 255)) for np_color_frame in np_color_frames]
    return colored_frames

def extract_frames_as_columns(frames):
    return [frame.resize((frame.width, 1), Image.ANTIALIAS) for frame in frames]

def create_row_from_frames(frames):
    widths, heights = zip(*(frame.size for frame in frames))
    total_width = sum(widths)
    max_height = max(heights)
    combined_image = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    for frame in frames:
        combined_image.paste(frame, (x_offset, 0))
        x_offset += frame.width
    return combined_image

# Paths to the GIF files
gif_filenames = ['raw', 'mask', 'edt']
gif_dir = "./data/mitosis-div-sample0/"
gif_paths = [gif_dir + filename + '.gif' for filename in gif_filenames]
# Process the GIF files
final_rows = []
for gif_path in gif_paths:
    blue_frames, yellow_frames = separate_channels(gif_path)
    blue_colored = apply_inferno_colormap(blue_frames)
    yellow_colored = apply_inferno_colormap(yellow_frames)
    blue_columns = extract_frames_as_columns(blue_colored)
    yellow_columns = extract_frames_as_columns(yellow_colored)
    final_rows.append(create_row_from_frames(blue_columns))
    final_rows.append(create_row_from_frames(yellow_columns))

# Combine rows to create the final image
final_width = max(row.width for row in final_rows)
final_height = sum(row.height for row in final_rows)
final_image = Image.new('RGB', (final_width, final_height))
y_offset = 0
for row in final_rows:
    final_image.paste(row, (0, y_offset))
    y_offset += row.height

# Save the final image
final_image.save('final_image.png')

# Optionally, display the final image
final_image.show()

C:\Users\dummy\AppData\Local\Temp\ipykernel_109360\1779290831.py:15: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  inferno = plt.cm.get_cmap("inferno")
C:\Users\dummy\AppData\Local\Temp\ipykernel_109360\1779290831.py:24: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  return [frame.resize((frame.width, 1), Image.ANTIALIAS) for frame in frames]
